In [1]:
import numpy as np
import pandas as pd
import sklearn
import pyterrier as pt
if not pt.started():
    pt.init()
pt.set_tqdm('tqdm')
import xml.etree.ElementTree as ET
import json
import random
import math
import os
import scipy as sp

PyTerrier 0.8.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
%run Pivot_Generation_Tools.ipynb

from ipynb.fs.full.Pivot_Generation_Tools import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


SyntaxError: invalid syntax (Pivot_Generation_Tools.ipynb, line 6)

In [4]:
def get_performance (run, queries, qrels, measure):
    #print(run, queries, qrels)
    
    e = pt.Experiment(
        [run],
        queries,
        qrels,
        eval_metrics=[measure],
    )
    
    #print(e[measure][0])
    return e[measure][0]

In [5]:
# Runs from other participants (official submissions)
runs_link_TRECDL  = "/home/mrim/galuscap/data/TRECDL-runs"

import os

primary_submissions_TRECDL = []

for link in os.listdir(runs_link_TRECDL):
    
    if (not ".gz" in link):
        continue
    
    full_link = runs_link_TRECDL + "/" + link
    run = pt.io.read_results(full_link)
    
    primary_submissions_TRECDL.append(run)

FileNotFoundError: [Errno 2] Aucun fichier ou dossier de ce type: '/home/mrim/galuscap/data/TRECDL-runs'

In [6]:
dataset_TRECDL = pt.get_dataset("irds:msmarco-passage-v2")
queries_TRECDL = pt.io.read_topics("/home/mrim/data/collection/msmarco_v2/topics/2022_queries.trec", format='trec')
qrels_TRECDL = pt.io.read_qrels("/home/mrim/data/collection/msmarco_v2/topics/2022.qrels.pass.withDupes.no1.txt")
# Qrels without duplicates !!!

In [ ]:
# Statistics of the submitted runs

map_TRECDL = get_runs_statistics(primary_submissions_TRECDL, queries_TRECDL, qrels_TRECDL, "map")
p10_TRECDL = get_runs_statistics(primary_submissions_TRECDL, queries_TRECDL, qrels_TRECDL, "P_10")
rr_TRECDL = get_runs_statistics(primary_submissions_TRECDL, queries_TRECDL, qrels_TRECDL, "recip_rank")
ndcg_TRECDL = get_runs_statistics(primary_submissions_TRECDL, queries_TRECDL, qrels_TRECDL, "ndcg")

print(map_TRECDL)
print(p10_TRECDL)
print(rr_TRECDL)
print(ndcg_TRECDL)

In [9]:
# Statistics of the document collection

TRECDL_stemmed_indexpath = "/home/mrim/data/collection/msmarco_v2/index/terrier_stemmed/"
TRECDL_stemmed_indexref = pt.IndexRef.of(os.path.join(TRECDL_stemmed_indexpath, "data.properties"))
TRECDL_stemmed_index = pt.IndexFactory.of(TRECDL_stemmed_indexref)
print(TRECDL_stemmed_index.getCollectionStatistics())

# XXX Neds to be changed to contain number of docs

docids_TRECDL = []
for i in range (0, 138364198):
    docid = TRECDL_stemmed_index.getMetaIndex().getAllItems(i)[0]
    docids_TRECDL.append(docid)

13:37:07.752 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 1 GiB of memory would be required.
13:37:07.771 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 8,6 GiB of memory would be required.
Number of documents: 138364198
Number of terms: 6816015
Number of postings: 3133342266
Number of fields: 1
Number of tokens: 3817081288
Field names: [text]
Positions:   false



In [ ]:
# Get distributions of the submitted runs

params_relevant_TRECDL, params_total_relevant_TRECDL, params_unjudged_TRECDL = get_distributions(qrels_TRECDL, queries_TRECDL, docids_TRECDL, primary_submissions_TRECDL, [], "run")

In [13]:
# Get distributions of the BM25 and TFIDF runs

tfidf_TRECDL = pt.BatchRetrieve(TRECDL_stemmed_indexref, wmodel="TF_IDF", properties={"termpipelines" : "Stopwords,PorterStemmer"})
bm25_TRECDL = pt.BatchRetrieve(TRECDL_stemmed_indexref, wmodel="BM25", properties={"termpipelines" : "Stopwords,PorterStemmer"})

tfidf_TRECDL_run = tfidf_TRECDL(queries_TRECDL)
bm25_TRECDL_run = bm25_TRECDL(queries_TRECDL)

pt.Experiment([tfidf_TRECDL_run, bm25_TRECDL_run], queries_TRECDL, qrels_TRECDL, eval_metrics=["map","P_10", "recip_rank", "ndcg"])

two_systems_TRECDL = [tfidf_TRECDL_run, bm25_TRECDL_run] 

params_relevant_TRECDL_2sys, params_total_relevant_TRECDL_2sys, params_unjudged_TRECDL_2sys = get_distributions(qrels_TRECDL, queries_TRECDL, docids_TRECDL, two_systems_TRECDL, [], "run")
print(params_relevant_TRECDL_2sys, params_total_relevant_TRECDL_2sys, params_unjudged_TRECDL_2sys)

17:35:24.349 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 1 GiB of memory would be required.
17:35:24.358 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 8,6 GiB of memory would be required.
17:35:37.445 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 1 GiB of memory would be required.
17:35:37.449 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 8,6 GiB of memory would be required.
2
            qid      docid          

/data1/home/mrim/gonzagab/envs/jup_env/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/data1/home/mrim/gonzagab/envs/jup_env/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check th

724 0.25 2 6
725 0.3 3 7
726 0.0 0 10
727 0.16666666666666666 1 5
728 0.2 1 4
729 0.0 0 2
730 0.14285714285714285 1 6
731 0.25 2 6
732 0.6666666666666666 2 1
733 0.4 2 3
734 0.16666666666666666 1 5
735 0.16666666666666666 1 5
736 0.0 0 7
737 0.0 0 5
738 0.0 0 8
739 0.14285714285714285 2 12
740 0.4 2 3
741 0.2222222222222222 2 7
742 0.3333333333333333 4 8
743 0.4 2 3
744 0.18181818181818182 2 9
745 0.21428571428571427 3 11
746 0.0 0 5
747 0.2857142857142857 2 5
748 0.2857142857142857 2 5
749 0.18181818181818182 2 9
750 0.0 0 7
751 0.0 0 5
752 0.0 0 4
753 0.2222222222222222 2 7
754 0.2857142857142857 2 5
755 0.0 0 7
756 0.3076923076923077 4 9
757 0.0 0 4
758 0.0 0 7
759 0.4444444444444444 4 5
760 0.0 0 10
761 0.0 0 8
762 0.1 1 9
763 0.25 2 6
764 0.3333333333333333 1 2
765 0.07692307692307693 1 12
766 0.1 1 9
767 0.0 0 7
768 0.375 3 5
769 0.0 0 4
770 0.16666666666666666 1 5
771 0.16666666666666666 1 5
772 0.125 1 7
773 0.1111111111111111 1 8
774 0.0 0 6
775 0.06666666666666667 1 14
776 0.

# Create the best and worst run estimate

### 1) First run the BM25 

In [ ]:
### 2) Get m, c, and t estimates

### 3) How much better can we expect, that the runs will be?

#### 3.1) Neural Systems

DuoBERT: At least 2x better (Table 21 in BERT book)
And Splade was even better

-> change m. It is linearly better, but the angle is ???




#### 3.2) Non-neural Systems

Which BM25 Do You Mean? A Large-Scale Reproducibility Study of Scoring Variants: cahnges are minimal

RM3: Table 11 (Jimmy Lin BERT book): 7% better

BM25 Parameters: http://www.cs.otago.ac.nz/homepages/andrew/papers/2014-2.pdf?ref=https://githubhelp.com
-x - up to 10%


# Applications

What we create is a simulation of the runs

We might need these:
- to be able to test user's models
- to be able to test cutoff methods (user models which 